In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin
import json
from youtube_transcript_api import YouTubeTranscriptApi
import string
import contractions
from sklearn.metrics import confusion_matrix, classification_report
from googleapiclient.discovery import build
import time
import os
import sys


In [ ]:
import requests

def get_mapmyindia_token(client_id, client_secret):
    url = "https://outpost.mapmyindia.com/api/security/oauth/token"
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(url, data=payload)
    data = response.json()
    return data["access_token"]
access_token = get_mapmyindia_token(ClientId,Client_Secret)

In [11]:
def get_coordinates_mapmyindia(place_name, token):
    search_url = f"https://atlas.mapmyindia.com/api/places/geocode"
    params = {"address": place_name}
    # params = {"": place_name}
    headers = {"Authorization": f"bearer {token}"}

    response = requests.get(search_url, params=params, headers=headers)
    data = response.json()
    return data
    if data.get("suggestedLocations"):
        loc = data["suggestedLocations"][0]
        return float(loc["latitude"]), float(loc["longitude"])
    
    return None, None


In [16]:
def get_lat_lon_from_eloc(eloc, token):
    url = f"https://atlas.mapmyindia.com/api/places/geocode?eloc={eloc}"
    headers = {
        "Authorization": f"bearer {token}"
    }
    response = requests.get(url, headers=headers)
    data = response.json()
    return data
    if "copResults" in data:
        lat = data["copResults"]["latitude"]
        lon = data["copResults"]["longitude"]
        return float(lat), float(lon)
    
    return None, None


In [17]:
access_token

'21588788-c221-4056-8c17-7354bf30a24f'

In [18]:
get_coordinates_mapmyindia('mubarak mahal, Jaipur, Rajasthan',access_token)


{'copResults': {'houseNumber': '',
  'houseName': '',
  'poi': 'Mubarak Mahal',
  'street': '',
  'subSubLocality': '',
  'subLocality': 'Janta Bazaar',
  'locality': 'Kanwar Nagar',
  'village': '',
  'subDistrict': 'Jaipur',
  'district': 'Jaipur District',
  'city': 'Jaipur',
  'state': 'Rajasthan',
  'pincode': '302002',
  'formattedAddress': 'Mubarak Mahal, Janta Bazaar, Kanwar Nagar, Jaipur, Jaipur District, Jaipur, Rajasthan, 302002',
  'eLoc': '8B2A6T',
  'geocodeLevel': 'poi',
  'confidenceScore': 0.4}}

In [19]:
get_lat_lon_from_eloc('8B2A6T',access_token)

{'error': 'Missing address parameter'}

### Key word Extractring 

In [ ]:
API_KEY = ''

In [ ]:
def get_video_statistics(video_id): 
    # video_id = '7cPLbiblb84'
    try:
        youtube = build('youtube', 'v3', developerKey=API_KEY)

        request = youtube.videos().list(
            part=['snippet','statistics'],
            id=video_id
        )
        response = request.execute()
        stats = response['items'][0]['statistics']
        viewCount = stats['viewCount'] if 'viewCount' in stats.keys() else 0
        likeCount = stats['likeCount'] if 'likeCount' in stats.keys() else 0
        commentCount = stats['commentCount'] if 'commentCount' in stats.keys() else 0 
        channelId = response['items'][0]['snippet']['channelId']
        channel_name = response['items'][0]['snippet']['channelTitle']

        print(response['items'][0]['statistics'],channelId,channel_name)
        
        return int(viewCount),int(likeCount),int(commentCount),channel_name,channelId
    except Exception as e:
        print(e)

def get_channel_statistics(channelId):
    try:
        youtube = build('youtube', 'v3', developerKey=API_KEY)
        request_channel =  youtube.channels().list(
            part=['statistics'],
            id=channelId
        )
        response_channel = request_channel.execute()
        subscriberCount= response_channel['items'][0]['statistics']['subscriberCount']
        return int(subscriberCount)
    except Exception as e:
        print(e)

def cleaning_sentence(text):
    text = text.lower()
    PUNCT_TO_REMOVE = string.punctuation
    ans = contractions.fix(text).translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    ans = ans.replace('music','')
    ans = ans.replace("  ",' ')
    ans = " ".join(ans.split())
    return ans


In [74]:
test_video_ids = [
    'p0MvovsCxCk',
    '7cPLbiblb84',
    'rm_j6O8y148',
    'Lv0PkSkKeSo',
    '5zA6OFpkPe0',
    'Oz18u64bM8I',
    'p0MvovsCxCk',
    '7cPLbiblb84',
    'Wt1LgJyF3s',
    'fP2zols1dag',
    'D1blpY-3ROE',
    'snwAYESRUEw',
    '48TcOu9kPqg',
    'dcRh1zTPnDQ',
    'lmhzpFfuWKc',
    '45qqtIpQP5M',
]

In [ ]:
root = os.getcwd()
model_path = os.path.join(root.split('notebooks')[0], 'models', 'ner_mod','model-best')
nlp_ner = spacy.load(model_path)
nlp     = spacy.load("en_core_web_lg")
ner     = dict()
    
for video_id in test_video_ids:
    try:
        print(video_id)
        
        viewCount,likeCount,commentCount,channel_name,channelId = get_video_statistics(video_id)
        subscriberCount = get_channel_statistics(channelId)

        root = os.getcwd()
        file_path = os.path.join(root.split('notebooks')[0], 'data', 'transcripts')

        file = open(f'{file_path}/transcript_{video_id}.txt',"r")
        file_c = open(f'{file_path}/transcript_{video_id}.txt',"r")
        file_string = file_c.read()
        for text in file:
            doc_ner = nlp_ner(cleaning_sentence(text)) 

            for ent in doc_ner.ents:
                # print(ent,'->', ent.label_)
                
                doc = nlp(ent.text)

                rating = file_string.count(ent.text) + ((viewCount+likeCount+commentCount+subscriberCount)/(subscriberCount))
        
                lemma = ' '.join([token.lemma_ for token in doc])
                if lemma in ner.keys():
                    # ner[ent.label_].append(ent.text)
                    ner[lemma]['ENT'].add(ent.label_)
                    ner[lemma]['rating'] = (rating+ner[lemma]['rating'])/2
                    ner[lemma]['suggested_by'].add(channel_name)
                    ner[lemma]['video_id'].add(video_id)
                else:
                    # ner[ent.label_] = [ent.text]
                    ner[lemma] = {
                        'word': ent.text,
                        'ENT': set([ent.label_]),
                        # 'keywords': list,
                        'suggested_by': set([channel_name]),
                        'rating': rating,
                        'video_id':set([video_id])
                    }
            # print('--x-x-x--x-x-x-x-x-x-x---x-x-x--')
    except Exception as e:
        print(e)
# tokkenization()
df = pd.DataFrame(ner)
df = df.T
df = df.reset_index().rename(columns={'index':'lemma_word'})

df['ENT_len'] = df['ENT'].apply(lambda x: len(list(x))) 
df['ENT_max'] = df['ENT'].apply(lambda x: (list(x))[0])

df[df['lemma_word']!=df['word']]
df.to_csv('D:/randomProjects/wanderly.ai/data/Parquet/all_locations_&_food.csv')
df_base_info = df[['lemma_word','word','suggested_by','video_id']]
df_entity_info = df[['lemma_word','rating','ENT_max']]

df_base_info = df_base_info.explode(['suggested_by','video_id']).drop_duplicates().reset_index()
del df_base_info['index']
df_base_info['key_word_extracted'] = 0

df_base_info.head(2)


p0MvovsCxCk
{'viewCount': '281734', 'favoriteCount': '0', 'commentCount': '97'} UCoCU-RvDqV1JgmbRgh79UTQ Singhsman 
7cPLbiblb84
{'viewCount': '2033200', 'likeCount': '16201', 'favoriteCount': '0', 'commentCount': '864'} UC_P_sA6Jf3iSsFueMwIP3vg TheSocialTraveller
rm_j6O8y148
{'viewCount': '192311', 'likeCount': '2535', 'favoriteCount': '0', 'commentCount': '93'} UCQ0X2x6lozB7qG30ctFeUiA Saturday Shooters
Lv0PkSkKeSo
{'viewCount': '117108', 'likeCount': '2218', 'favoriteCount': '0', 'commentCount': '188'} UC2N5r2FvEOiDPKvaLW9rivw Aniruddha Patil
5zA6OFpkPe0
{'viewCount': '10204', 'likeCount': '201', 'favoriteCount': '0', 'commentCount': '33'} UCCq9CLWwVP4fdYy2N_ypTjg Sisters vs Globe
Oz18u64bM8I
{'viewCount': '9933', 'likeCount': '170', 'favoriteCount': '0', 'commentCount': '36'} UC-w8ULGFYLrjbdsZ6twWO3Q Travel Tales
p0MvovsCxCk
{'viewCount': '281734', 'favoriteCount': '0', 'commentCount': '97'} UCoCU-RvDqV1JgmbRgh79UTQ Singhsman 
7cPLbiblb84
{'viewCount': '2033200', 'likeCount': '16201

,lemma_word,word,suggested_by,video_id,key_word_extracted
0,sindhi dal,sindhi dal,Singhsman,p0MvovsCxCk,0
1,kudi chhola,kudi chhola,Singhsman,p0MvovsCxCk,0


In [102]:
# df_base_info = df[['lemma_word','word','suggested_by','video_id']]
# df_entity_info = df[['lemma_word','rating','ENT_max']]

# df_base_info = df_base_info.explode(['suggested_by','video_id']).drop_duplicates().reset_index()
# del df_base_info['index']
# df_base_info['key_word_extracted'] = 0
# df_base_info.head(2)
df = pd.read_csv('D:/randomProjects/wanderly.ai/data/Parquet/all_locations_&_food.csv')
df_base_info = df[['lemma_word','word','suggested_by','video_id']]
df_entity_info = df[['lemma_word','rating','ENT_max']]

df_base_info = df_base_info.explode(['suggested_by','video_id']).drop_duplicates().reset_index()
del df_base_info['index']
df_base_info['key_word_extracted'] = 0

df_base_info.head(2)


,lemma_word,word,suggested_by,video_id,key_word_extracted
0,sindhi dal,sindhi dal,{'Singhsman '},{'p0MvovsCxCk'},0
1,kudi chhola,kudi chhola,{'Singhsman '},{'p0MvovsCxCk'},0


In [103]:
# df_base_info.explode(['suggested_by','video_id'])
df_base_info['video_id'] = df_base_info[df_base_info['video_id'].notna()]['video_id'].apply(lambda x :(x.replace("{",'').replace("}",'').replace("'",'').split(", ")))
df_base_info['suggested_by'] = df_base_info[df_base_info['suggested_by'].notna()]['suggested_by'].apply(lambda x :(x.replace("{",'').replace("}",'').replace("'",'').split(", ")))

df_base_info = df_base_info.explode(['suggested_by','video_id']).drop_duplicates().reset_index()
del df_base_info['index']
df_base_info['key_word_extracted'] = 0
df_base_info


,lemma_word,word,suggested_by,video_id,key_word_extracted
0,sindhi dal,sindhi dal,Singhsman,p0MvovsCxCk,0
1,kudi chhola,kudi chhola,Singhsman,p0MvovsCxCk,0
2,rice,rice,Singhsman,p0MvovsCxCk,0
3,hai pri,hai pri,Singhsman,p0MvovsCxCk,0
4,tamarind sauce,tamarind sauce,Singhsman,p0MvovsCxCk,0
...,...,...,...,...,...
477,lane tripolia,lane tripolia,Sakshi Yadav,lmhzpFfuWKc,0
478,agar,agar,Sakshi Yadav,lmhzpFfuWKc,0
479,pardesh manya,pardesh manya,Sakshi Yadav,lmhzpFfuWKc,0
480,bapu,bapu,Sakshi Yadav,lmhzpFfuWKc,0


In [104]:
df_base_info.shape,df.shape

((482, 5), (388, 9))

In [105]:
def get_context_window(id, text, window_size=50):
    root = os.getcwd().split('wanderly.ai')[0]
    file_path = os.path.join(root, f'wanderly.ai/data/transcripts/')
    file = open(file_path + f'transcript_{id}.txt',"r")
    
    file_string = file.read()
    print(file_path + f'transcript_{id}.txt')
    # loc = 'city palace'
    start = file_string.find(text)
    end = file_string.rfind(text)
    context_text = (file_string[start-window_size:end+window_size])
    return context_text

In [106]:
df_base_info['details'] = df_base_info.apply(lambda x: get_context_window(x['video_id'],x['word'],100),axis=1)


d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_7cPLbiblb84.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_Lv0PkSkKeSo.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_rm_j6O8y148.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_p0MvovsCxCk.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_5zA6OFpkPe0.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_fP2zols1dag.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_snwAYESRUEw.txt
d:\randomProjects\wanderly.ai/data/transcripts/transcript_48TcOu9kPqg.txt
d:\randomProjects\wanderly.ai/data/tra

In [ ]:

nlp = spacy.load("en_core_web_lg")
def process_text(text):
    """
    Removes stop words and lemmatizes the input text.

    Args:
        text (str): The input text.

    Returns:
        str: The processed text with stop words removed and lemmatized.
    """
    
    doc = nlp(text)
    filtered_and_lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop and len(token.text)>2]
    index = 1
    while index<len(filtered_and_lemmatized_tokens):
        if filtered_and_lemmatized_tokens[index] == filtered_and_lemmatized_tokens[index-1]:
            del filtered_and_lemmatized_tokens[index-1]
        else:
            index = index+1

    return " ".join(filtered_and_lemmatized_tokens).replace('\n','')


In [108]:
df_base_info['clean_test'] = df_base_info['details'].apply(process_text)
df_base_info


,lemma_word,word,suggested_by,video_id,key_word_extracted,details,clean_test
0,sindhi dal,sindhi dal,Singhsman,p0MvovsCxCk,0,n have come well in the morning time very much...,come morning time breakfast center breakfast g...
1,kudi chhola,kudi chhola,Singhsman,p0MvovsCxCk,0,st of our sindhis you have heard that dal dish...,sindhis hear dal dish sindhis people feel good...
2,rice,rice,Singhsman,p0MvovsCxCk,0,at dal dish in sindhis people feel very good a...,dal dish sindhis people feel good absolutely k...
3,hai pri,hai pri,Singhsman,p0MvovsCxCk,0,i rice is ok these three or four things are ma...,rice thing maintain morning pricing hai pri si...
4,tamarind sauce,tamarind sauce,Singhsman,p0MvovsCxCk,0,add ok so the lentils go but you have a little...,add lentil little bit chickpea onion tamarind ...
...,...,...,...,...,...,...,...
477,lane tripolia,lane tripolia,Sakshi Yadav,lmhzpFfuWKc,0,you will find hawa mahal and in the hand side ...,find hawa mahal hand left jori bazaar call lan...
478,agar,agar,Sakshi Yadav,lmhzpFfuWKc,0,d there i did not do the bark there how cannot...,bark look good mirror good lagar hai 450 last ...
479,pardesh manya,pardesh manya,Sakshi Yadav,lmhzpFfuWKc,0,o now we have come to jaipurs main work get\na...,come jaipurs main work miss turban tie come de...
480,bapu,bapu,Sakshi Yadav,lmhzpFfuWKc,0,age\nwell\nbecause there is no many markets h...,age market link confusing bapu bazaar jauri ma...


In [109]:
df_base_info = df_base_info.merge(df_entity_info,on=['lemma_word'],how='inner')
landmarks = df_base_info[df_base_info['ENT_max'].isin(['LANDMARK','AREA'])].copy().reset_index().drop(columns='index',axis=1)
food = df_base_info[df_base_info['ENT_max'].isin(['FOOD','FOOD SHOP'])].copy().reset_index().drop(columns='index',axis=1)
# df_base_info = df_base_info[df_base_info['ENT_max'].isin(['LANDMARK','AREA','FOOD','FOOD SHOP'])].copy().reset_index().drop(columns='index',axis=1)

In [110]:
df_base_info = df_base_info[df_base_info['ENT_max'].isin(['LANDMARK','AREA','FOOD','FOOD SHOP'])].copy().reset_index().drop(columns='index',axis=1)

In [111]:
# from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer(stop_words='english',ngram_range=(1,2),min_df=0.28)
# dim = cv.fit_transform(food['clean_test'])
# cv_df = pd.DataFrame(dim.toarray(),columns=cv.get_feature_names_out())
# print(cv_df.columns)
# cv_df

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

root = os.getcwd()
model_path = os.path.join(root.split('notebooks')[0], 'models')


tvid_loc = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=0.2,min_df=0.05)
X_locations = tvid_loc.fit_transform(landmarks['clean_test'])
tvid_loc_df = pd.DataFrame(X_locations.toarray(),columns=tvid_loc.get_feature_names_out())

with open(model_path+'/tvid_loc.pkl', 'wb') as file:
    # Serialize and save the model to the file
    pickle.dump(tvid_loc, file)


tvid_food = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=0.3,min_df=0.15)
X_food = tvid_food.fit_transform(food['clean_test'])
tvid_food_df = pd.DataFrame(X_food.toarray(),columns=tvid_food.get_feature_names_out())

with open(model_path+'/tvid_food.pkl', 'wb') as file:
    # Serialize and save the model to the file
    pickle.dump(tvid_food, file)


# tvid_keyWords = TfidfVectorizer(stop_words='english',ngram_range=(1,2),min_df=0.15)
# X_keyWords = tvid_keyWords.fit_transform(df_base_info['clean_test'])
# tvid_keyWords_df = pd.DataFrame(X_keyWords.toarray(),columns=tvid_keyWords.get_feature_names_out())

print(tvid_food_df.columns)
print(tvid_loc_df.columns)
# print(tvid_keyWords_df.columns)

# df_base_info = pd.concat([df_base_info,tvid_keyWords_df],axis=1)
# df_base_info.head(2)

Index(['actually', 'amazing', 'bada', 'bhandar', 'big', 'bit', 'chaat',
       'cheese', 'chilli', 'chocolate', 'chutney', 'cream', 'day', 'different',
       'eat', 'enjoy', 'famous', 'feel', 'flavor', 'flour', 'food', 'friend',
       'garlic', 'ghee', 'hot', 'ice', 'ice cream', 'inside', 'kachori',
       'kind', 'know', 'lassi', 'let', 'light', 'little', 'little bit', 'look',
       'lot', 'love', 'mahavir', 'mahavir rabri', 'masala', 'milk', 'mirchi',
       'nice', 'old', 'onion', 'paneer', 'people', 'plate', 'popular',
       'potato', 'rabri', 'rajasthani', 'restaurant', 'samosa', 'shop',
       'simple', 'size', 'small', 'special', 'specialty', 'start', 'sweet',
       'taste', 'tell', 'think', 'time', 'variety', 'want', 'year'],
      dtype='object')
Index(['200', '400', '500', '900', 'absolutely', 'accord', 'address', 'albert',
       'albert hall', 'amazing',
       ...
       'wind', 'window', 'work', 'world', 'world large', 'worth', 'year',
       'year old', 'yeh', 'yes'

In [128]:
# H = pd.DataFrame(H,index=['Snaks','Evening','Desert','Tikka'])
# W = pd.DataFrame(W,columns=['Snaks','Evening','Desert','Tikka'])

In [129]:
model_path

'd:\\randomProjects\\wanderly.ai\\models'

In [130]:
X_food

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2261 stored elements and shape (161, 71)>

In [131]:
from sklearn.decomposition import NMF
nmf_model_loc = NMF(n_components=7)
W_locations = nmf_model_loc.fit_transform(X_locations)
H_locations = nmf_model_loc.components_

nmf_model_food = NMF(n_components=4)
W_food = nmf_model_food.fit_transform(X_food)
H_food = nmf_model_food.components_

with open(model_path+'/nmf_model_loc.pkl', 'wb') as file:
    # Serialize and save the model to the file
    pickle.dump(nmf_model_loc, file)

with open(model_path+'/nmf_model_food.pkl', 'wb') as file:
    # Serialize and save the model to the file
    pickle.dump(nmf_model_food, file)


def display_topic(H,tvid):
    for topic_num, topic_array in enumerate(H):
        top_features = topic_array.argsort()[::-1][:10]
        top_words = [tvid.get_feature_names_out()[i] for i in top_features]
        print(f"Topic {topic_num +1}: {top_words}")

display_topic(H_locations,tvid_loc)
display_topic(H_food,tvid_food)
# H = pd.DataFrame(H,index=['Forts','Generic','Temple','Temples & Architectur','Scientific Monuments','Lakeside Forts','Views'])
W_locations = pd.DataFrame(W_locations,columns=['Forts','Generic','Temple','Temples & Architectur','Scientific Monuments','Lakeside Forts','Views'])

# H = pd.DataFrame(H,index=['Forts','Generic','Temple','Temples & Architectur','Scientific Monuments','Lakeside Forts','Views'])
W_food = pd.DataFrame(W_food,columns=['Snaks','Evening','Desert','Tikka'])

food = pd.concat([food,W_food],axis=1)
landmarks = pd.concat([landmarks,W_locations],axis=1)

df_base_info = pd.concat([landmarks,food]).fillna(0)
df_base_info.head(2)


Topic 1: ['restaurant', 'experience', 'different', 'eat', 'shop', 'coffee', 'thing', 'food', 'plate', 'kachori']
Topic 2: ['gate', 'bazaar', 'market', 'bapu', 'left', 'bag', 'right', 'new', 'tell', 'shop']
Topic 3: ['amer', 'build', 'amer fort', 'courtyard', 'king', 'jaigarh', 'jaigarh fort', 'nahargarh', 'entry', 'sheesh']
Topic 4: ['hawa', 'hawa mahal', 'floor', 'window', 'second', 'know', 'royal', 'maharaja', 'city palace', 'singh']
Topic 5: ['mandir', 'park', 'ganesh', 'visit jaipur', 'white', 'world', 'gate', 'deewane', 'jawahar', 'deewane khas']
Topic 6: ['garden', 'museum', 'hall museum', 'albert hall', 'albert', 'hall', 'apart', 'roam', 'build', 'know']
Topic 7: ['sun', 'jantar', 'mantar', 'city', 'jantar mantar', 'lake', 'water', 'view', 'jaipur city', 'jal']
Topic 1: ['masala', 'think', 'feel', 'time', 'bit', 'little', 'lot', 'little bit', 'different', 'actually']
Topic 2: ['kachori', 'samosa', 'sweet', 'onion', 'mirchi', 'shop', 'bada', 'big', 'famous', 'lassi']
Topic 3: ['y

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [132]:
tvid_keyWords = TfidfVectorizer(stop_words='english',ngram_range=(1,2),min_df=0.15)
X_keyWords = tvid_keyWords.fit_transform(df_base_info['clean_test'])
tvid_keyWords_df = pd.DataFrame(X_keyWords.toarray(),columns=tvid_keyWords.get_feature_names_out())

print(tvid_keyWords_df.columns)

root = os.getcwd()
model_path = os.path.join(root.split('notebooks')[0], 'models')
with open(model_path+'/tvid_keyWords.pkl', 'wb') as file:
    # Serialize and save the model to the file
    pickle.dump(tvid_keyWords, file)


Index(['big', 'come', 'day', 'different', 'eat', 'famous', 'feel', 'food',
       'fort', 'good', 'inside', 'jaipur', 'kachori', 'know', 'let', 'like',
       'little', 'look', 'lot', 'mahal', 'old', 'palace', 'people', 'place',
       'right', 'shop', 'small', 'special', 'temple', 'thing', 'time', 'try',
       'visit', 'want', 'year'],
      dtype='object')


In [133]:
df_base_info = df_base_info.reset_index().drop(['index'],axis=1)

In [118]:
df_base_info = pd.concat([df_base_info,tvid_keyWords_df],axis=1)
df_base_info.head(2)

,lemma_word,word,suggested_by,video_id,key_word_extracted,details,clean_test,rating,ENT_max,Forts,...,shop,small,special,temple,thing,time,try,visit,want,year
0,hai pri,hai pri,Singhsman,p0MvovsCxCk,0,i rice is ok these three or four things are ma...,rice thing maintain morning pricing hai pri si...,19.837405,LANDMARK,0.084501,...,0.0,0.0,0.0,0.0,0.915543,0.0,0.00000,0.0,0.0,0.0
1,allredi trai,allredi trai,Singhsman,p0MvovsCxCk,0,e lentil and of tapre mirchi will keep my effo...,lentil tapre mirchi effort basically let thing...,19.837405,LANDMARK,0.072218,...,0.0,0.0,0.0,0.0,0.476420,0.0,0.51575,0.0,0.0,0.0


In [119]:
# landmarks = pd.concat([landmarks,W],axis=1)


In [120]:
# food = pd.concat([food,W],axis=1)
# landmarks = pd.concat([landmarks,W],axis=1)
# df_base_info = pd.concat([landmarks,food]).fillna(0)
# df_base_info.head(2)

In [121]:
# df_base_info = pd.concat([landmarks,food]).fillna(0)
# df_base_info.head(2)

In [122]:
# def normalize_rateing(max_r,min_r,current_r):
#     return (current_r - min_r)/(max_r + min_r)

In [123]:
min_max_category_rateing = df_base_info.groupby(by='ENT_max')['rating'].agg(['min','max']).reset_index().rename(columns={'min':'min_category_rating','max':'max_category_rating'})
min_max_category_rateing

,ENT_max,min_category_rating,max_category_rating
0,AREA,2.372318,43.837405
1,FOOD,2.660633,35.610902
2,FOOD SHOP,2.456626,35.610902
3,LANDMARK,2.372318,39.610902


In [124]:
df_base_info = df_base_info.merge(min_max_category_rateing,on='ENT_max')
df_base_info['normalize_rating'] = (df_base_info['rating'] - df_base_info['min_category_rating'])/(df_base_info['max_category_rating'] + df_base_info['min_category_rating'])
df_base_info.drop(['min_category_rating','max_category_rating'],axis=1,inplace=True)
df_base_info


,lemma_word,word,suggested_by,video_id,key_word_extracted,details,clean_test,rating,ENT_max,Forts,...,small,special,temple,thing,time,try,visit,want,year,normalize_rating
0,hai pri,hai pri,Singhsman,p0MvovsCxCk,0,i rice is ok these three or four things are ma...,rice thing maintain morning pricing hai pri si...,19.837405,LANDMARK,0.084501,...,0.000000,0.000000,0.0,0.915543,0.000000,0.000000,0.000000,0.000000,0.0,0.416002
1,allredi trai,allredi trai,Singhsman,p0MvovsCxCk,0,e lentil and of tapre mirchi will keep my effo...,lentil tapre mirchi effort basically let thing...,19.837405,LANDMARK,0.072218,...,0.000000,0.000000,0.0,0.476420,0.000000,0.515750,0.000000,0.000000,0.0,0.416002
2,allredi,allredi,Singhsman,p0MvovsCxCk,0,e lentil and of tapre mirchi will keep my effo...,lentil tapre mirchi effort basically let thing...,20.837405,AREA,0.072218,...,0.000000,0.000000,0.0,0.442900,0.000000,0.479462,0.000000,0.000000,0.0,0.399593
3,nana ji,nana ji,Singhsman,p0MvovsCxCk,0,name sir gulshan bhatia sir our father started...,sir gulshan bhatia sir father start paneer goo...,19.837405,LANDMARK,0.044190,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.416002
4,vaccine,vaccines,Singhsman,p0MvovsCxCk,0,something when the pure cheese was with you a...,pure cheese blessing cheese introduction vacci...,19.837405,AREA,0.049754,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.377953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,vaidyashala,vaidyashala,Yatra Mitra,dcRh1zTPnDQ,0,was on this lovely vaidyalaya by included in ...,lovely vaidyalaya include world heritage list ...,4.017759,FOOD,0.000000,...,0.000000,0.000000,0.0,0.000000,0.311179,0.000000,0.000000,0.341664,0.0,0.035460
443,ganesh poll,ganesh poll,Yatra Mitra,dcRh1zTPnDQ,0,ve also been shot here see you very special vi...,shoot special view look view ganesh poll win h...,3.017759,FOOD SHOP,0.000000,...,0.000000,0.743404,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.014740
444,rajo rani,rajo rani,Sakshi Yadav,lmhzpFfuWKc,0,s\nappreciation explore ok such bags find a lo...,appreciation explore bag find lot bag like bag...,2.456626,FOOD SHOP,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.503067,0.0,0.000000
445,rahi,rahi,Sakshi Yadav,lmhzpFfuWKc,0,ight prints show the dress that took the dress...,ight print dress take dress like dear brother ...,2.456626,FOOD SHOP,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [125]:
df_base_info.columns

Index(['lemma_word', 'word', 'suggested_by', 'video_id', 'key_word_extracted',
       'details', 'clean_test', 'rating', 'ENT_max', 'Forts', 'Generic',
       'Temple', 'Temples & Architectur', 'Scientific Monuments',
       'Lakeside Forts', 'Views', 'Snaks', 'Evening', 'Desert', 'Tikka', 'big',
       'come', 'day', 'different', 'eat', 'famous', 'feel', 'food', 'fort',
       'good', 'inside', 'jaipur', 'kachori', 'know', 'let', 'like', 'little',
       'look', 'lot', 'mahal', 'old', 'palace', 'people', 'place', 'right',
       'shop', 'small', 'special', 'temple', 'thing', 'time', 'try', 'visit',
       'want', 'year', 'normalize_rating'],
      dtype='object')

In [126]:
df_base_info.to_csv('D:/randomProjects/wanderly.ai/data/Parquet/final_loc_food_df.csv',index=False)

In [100]:
req_col = [i for i in df_base_info.columns if i not in ['lemma_word', 'word', 'suggested_by', 
                                                        'video_id','key_word_extracted','details', 
                                                        'clean_test', 'rating', 'ENT_max','normalize_rating']]

In [101]:
X = df_base_info[req_col]
Y = df_base_info[['lemma_word']]

In [50]:
'''
What excites you most about traveling?
Which of these travel quotes resonates with you the most?
If you had a 3-day break, what would you do?
What type of destinations do you find most appealing?
What kind of experiences do you look for while traveling?
What do you often post or talk about after your trips?
Which of these describe you best when traveling?
What's the first thing you usually do after reaching a new place?
Describe your dream travel day in 2–3 sentences.
'''

"\nWhat excites you most about traveling?\nWhich of these travel quotes resonates with you the most?\nIf you had a 3-day break, what would you do?\nWhat type of destinations do you find most appealing?\nWhat kind of experiences do you look for while traveling?\nWhat do you often post or talk about after your trips?\nWhich of these describe you best when traveling?\nWhat's the first thing you usually do after reaching a new place?\nDescribe your dream travel day in 2–3 sentences.\n"

In [51]:
user_input = ['I like to trave to find me, enjoy, capture the movement',
'Like to explore the culture, heritage and local food',
'relax chill with friends',
'peacefull place calm and relaxing',
'chill good local food instagram worthy locations, vloging',
'Explorer – I love wandering and getting lost',
'Explore the surroundings on foot',
'Dream travel worule be lost in the movement, relax, local food, chill vibes, haritage, mountain person but \
also like beaches and water side resorts']


In [52]:
user_input = ' '.join(user_input)
print(user_input)

I like to trave to find me, enjoy, capture the movement Like to explore the culture, heritage and local food relax chill with friends peacefull place calm and relaxing chill good local food instagram worthy locations, vloging Explorer – I love wandering and getting lost Explore the surroundings on foot Dream travel worule be lost in the movement, relax, local food, chill vibes, haritage, mountain person but also like beaches and water side resorts


In [53]:
user = pd.DataFrame({'name':['Jatin'],'desc':[user_input]})
user

,name,desc
0,Jatin,"I like to trave to find me, enjoy, capture the..."


In [54]:
user['clean_dec'] = user['desc'].apply(lambda x: process_text(cleaning_sentence(x)))
user

,name,desc,clean_dec
0,Jatin,"I like to trave to find me, enjoy, capture the...",like trave find enjoy capture movement like ex...


In [55]:
X_locations = tvid_loc.transform(user['clean_dec'])
tvid_loc_df = pd.DataFrame(X_locations.toarray(),columns=tvid_loc.get_feature_names_out())

X_food = tvid_food.transform(user['clean_dec'])
tvid_food_df = pd.DataFrame(X_food.toarray(),columns=tvid_food.get_feature_names_out())

X_keyWords = tvid_keyWords.transform(user['clean_dec'])
tvid_keyWords_df = pd.DataFrame(X_keyWords.toarray(),columns=tvid_keyWords.get_feature_names_out())
# tvid_keyWords_df = pd.DataFrame(X_keyWords.toarray(),columns=tvid_keyWords.get_feature_names_out())



In [56]:
tvid_keyWords_df

,big,come,day,different,eat,famous,feel,food,fort,good,...,shop,small,special,temple,thing,time,try,visit,want,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77257,0.0,0.195676,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
W_locations = nmf_model_loc.transform(X_locations)
H_locations = nmf_model_loc.components_

W_food = nmf_model_food.transform(X_food)
H_food = nmf_model_food.components_

W_locations = pd.DataFrame(W_locations,columns=['Forts','Generic','Temple','Temples & Architectur','Scientific Monuments','Lakeside Forts','Views'])

# H = pd.DataFrame(H,index=['Forts','Generic','Temple','Temples & Architectur','Scientific Monuments','Lakeside Forts','Views'])
W_food = pd.DataFrame(W_food,columns=['Snaks','Evening','Desert','Tikka'])


In [58]:
type(W_locations)

pandas.core.frame.DataFrame

In [59]:
type(W_food)

pandas.core.frame.DataFrame

In [60]:
user_transform = pd.concat([user,tvid_keyWords_df,W_locations,W_food],axis=1)
user_transform

,name,desc,clean_dec,big,come,day,different,eat,famous,feel,...,Generic,Temple,Temples & Architectur,Scientific Monuments,Lakeside Forts,Views,Snaks,Evening,Desert,Tikka
0,Jatin,"I like to trave to find me, enjoy, capture the...",like trave find enjoy capture movement like ex...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003722,0.015045,0.0,0.0,0.0,0.062283,0.0,0.0,0.182176,0.0


In [61]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

def get_similar_objects(check,X,Y,top_n=5):
    common = [i for i in X.columns if i in check.columns]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X[common])
    user_scaled = scaler.transform(check[common])
    
    # Compute cosine similarity between user vector and all item vectors
    similarity_scores = cosine_similarity(user_scaled, X[common])[0]

    # Add scores to DataFrame
    Y['similarity'] = similarity_scores

    # Sort top recommendations
    top_recommendations = Y.sort_values(by='similarity', ascending=False).head(10)
    return top_recommendations

In [62]:
get_similar_objects(user_transform,X,Y,10)

C:\Users\Jatin Arora\AppData\Local\Temp\ipykernel_30724\2801194235.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['similarity'] = similarity_scores


,lemma_word,similarity
7,sodala,0.712143
188,gaurav tar,0.674760
166,shweta,0.620578
301,gulab ji,0.596382
370,mahavir rabri bhandar,0.553632
228,tride,0.533788
233,mi road,0.531921
323,dhaba,0.526955
426,rawat mishthan bhandar,0.509835
338,aamir food,0.494557


In [63]:
df_base_info[df_base_info['lemma_word']=='sambar lake']

,lemma_word,word,suggested_by,video_id,key_word_extracted,details,clean_test,rating,ENT_max,Forts,...,small,special,temple,thing,time,try,visit,want,year,normalize_rating
153,sambar lake,sambar lake,Aniruddha Patil,Lv0PkSkKeSo,0,m jaipur a distance of 75 km but indias bigges...,jaipur distance india big salt settle water la...,6.372318,LANDMARK,0.023057,...,0.208831,0.214811,0.0,0.0,0.0,0.0,0.0,0.0,0.431709,0.095276


In [ ]:
get_similar_objects(df_base_info[df_base_info['lemma_word']=='chowki dhani'],X,Y,5)

C:\Users\Jatin Arora\AppData\Local\Temp\ipykernel_9820\2013376519.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['similarity'] = similarity_scores


,lemma_word,similarity
40,chowki dhani,1.000000
7,sodala,0.825258
166,dhaba,0.771376
56,nahargarh,0.639082
8,gulab ji,0.581485
153,shankar samosa,0.564242
6,shimla tutter,0.540364
177,maggi,0.537956
16,hawa mahal,0.537956
141,tamarind sauce,0.523520


In [ ]:
top_recommendations = Y.sort_values(by='similarity', ascending=False).head(10)
top_recommendations

,lemma_word,similarity
27,pune,0.656606
85,zenana deodi,0.613180
9,nana ji,0.607636
49,mubarak mahal,0.546297
180,chaat,0.546149
50,palace of reception,0.519179
195,arun bhai,0.513964
28,hawa mahal,0.482137
178,raman dosa,0.380070
57,amer fort,0.370288


In [ ]:
# Re-import required modules after environment reset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Define the main function to recommend keywords
def recommend_keywords_for_user(
    user_input,
    df_base_info,
    tfidf_models,
    nmf_models,
    keyword_vectorizer,
    process_text_func,
    req_col,
    topic_labels,
    normalize=True,
    top_n=10
):
    """
    Full keyword recommendation pipeline from raw user input.

    Args:
        user_input (str): Raw user description or answers.
        df_base_info (pd.DataFrame): DataFrame containing features and metadata.
        tfidf_models (dict): Dictionary with 'location' and 'food' TfidfVectorizers.
        nmf_models (dict): Dictionary with 'location' and 'food' NMF models.
        keyword_vectorizer: TF-IDF vectorizer used for general keywords.
        process_text_func (callable): Text cleaning and lemmatization function.
        req_col (list): Final column names used in cosine similarity.
        topic_labels (dict): {'location': [...], 'food': [...]}
        normalize (bool): Whether to scale all features.
        top_n (int): Number of recommendations to return.

    Returns:
        pd.DataFrame: Top N recommended keywords/entities.
    """
    # 1. Clean and prepare user input
    user_clean = process_text_func(user_input)

    # 2. Transform into TF-IDF vectors
    X_loc = tfidf_models['location'].transform([user_clean])
    X_food = tfidf_models['food'].transform([user_clean])
    X_key = keyword_vectorizer.transform([user_clean])

    # 3. Apply NMF topic models
    W_loc = nmf_models['location'].transform(X_loc)
    W_food = nmf_models['food'].transform(X_food)

    # 4. Build user feature vector
    user_vector = np.concatenate([
        W_loc[0],
        W_food[0],
        [0.5],  # normalize_rating neutral
        X_key.toarray()[0][:len(req_col) - (len(W_loc[0]) + len(W_food[0]) + 1)]
    ])
    user_vector = pd.DataFrame([user_vector], columns=req_col[:len(user_vector)])

    # 5. Feature scaling (optional)
    if normalize:
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(df_base_info[req_col])
        user_scaled = scaler.transform(user_vector)
    else:
        X_scaled = df_base_info[req_col].values
        user_scaled = user_vector.values

    # 6. Cosine similarity
    similarity_scores = cosine_similarity(user_scaled, X_scaled)[0]
    df_result = df_base_info.copy()
    df_result['score'] = similarity_scores

    # 7. Return top N
    return df_result.sort_values(by='score', ascending=False).head(top_n)[
        ['lemma_word', 'ENT_max', 'video_id', 'suggested_by', 'score']
    ]
